-------FINAL VERSION BELOW--------------

In [21]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

In [3]:
url = "https://us.shein.com/style/Women-Clothing-sc-001121425.html?ici=us_tab01navbar05&src_module=topcat&src_tab_page_id=page_home1690914418587&src_identifier=fc%3DWomen%60sc%3DCLOTHING%60tc%3D0%60oc%3D0%60ps%3Dtab01navbar05%60jc%3DitemPicking_001121425&srctype=category&userpath=category-CLOTHING&sort=7"

In [2]:
HEADERS = ({'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36','Accept-Language': 'en-US, en;q=0.5'})

In [4]:
webpage = requests.get(url, headers = HEADERS)
webpage #200 works

<Response [200]>

In [5]:
soup = BeautifulSoup(webpage.content, "html.parser")

In [6]:
links = soup.find_all("a", attrs={'class':'S-product-item__img-container j-expose__product-item-img'})

In [27]:
def scrap_data(url,products_data):
    ###############################################
    # GOOOOOOOD TO GO !!!!!!!!! (for product page)
    ###############################################
    
    driver = webdriver.Chrome()  # Replace with your browser driver (e.g., ChromeDriver)
    driver.get(url)
    WebDriverWait(driver, 10)
    
    # Wait for the pop-up window to load
    WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.XPATH,'//i[@class="iconfont icon-close she-close"]')))

    # Click button(close pop-up window)
    close_button = driver.find_element(By.XPATH,'//i[@class="iconfont icon-close she-close"]')
    close_button.click()
    
    # Scroll to bottom
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

    # Wait for 10 seconds for the elements to load
    WebDriverWait(driver, 10)

    # Extract product name, price, discount, average rating, tags, and top 100 reviews
    product_name = driver.find_element(By.XPATH, '//h1[@class="product-intro__head-name"]').text
    
    #print("GOT " + product_name + "!")
    
    try:
        product_price = driver.find_element(By.CLASS_NAME, 'product-intro__head-mainprice').text
    except NoSuchElementException:
        product_price = driver.find_element(By.CLASS_NAME, 'del-price').text
    #href = element.find_element(By.XPATH, "./a["href"]")

    # Extract discount if available
    # could not read somehow 
    #----- TODO: --------------------
    try:
        product_discount = driver.find_element(By.CLASS_NAME, 'discount-label').text
    except NoSuchElementException:
        product_discount = 0
            
    average_rating = driver.find_element(By.XPATH, '//div[@class="rate-num"]').text
    fit_ratings = driver.find_elements(By.XPATH, '//div[@class="fit-item"]')
    fit_rate = [fit_rating.text for fit_rating in fit_ratings]
    num_review = driver.find_element(By.CLASS_NAME, 'common-reviews__tab-item_active').text
    #product_tags = driver.find_element(By.CLASS_NAME, 'common-reviews__tag').text
    product_tags = driver.find_elements(By.CLASS_NAME, 'common-reviews__tag')
    product_tag_list = [product_tag.text for product_tag in product_tags]    
        
    # Grab reviews
    reviews_elems = driver.find_elements(By.CLASS_NAME, 'rate-des')
    top_3_reviews = [review_elem.text for review_elem in reviews_elems]

    products_data.append({
        'product_name': product_name,
        'product_price': product_price,
        'product_discount': product_discount,
        'average_rating': average_rating,
        'fit_rating': fit_rate,
        'num_review': num_review,
        'product_tags': product_tag_list,
        'top_3_reviews': top_3_reviews
    })
    
    print("GOT "+product_name+".\n")

    driver.quit()
    return products_data

In [28]:
if __name__ == "__main__":
    products_data = []
    for i in range(len(links)):
        print("starting %sth iteration."% i)
        link = links[i].get('href')
        products_data = scrap_data("https://us.shein.com" + link, products_data)
        
    output = pd.DataFrame(products_data)
    output.to_csv("shein_women_clothing_first_3.csv")

starting 0th iteration.
GOT Button Front Denim Skirt.
starting 1th iteration.
GOT SHEIN SXY Solid High Waist Leggings.
starting 2th iteration.
GOT SHEIN EZwear 3pcs Space Dye Drawstring Slant Pocket Sweatpants.
starting 3th iteration.
GOT SHEIN Essnce Floral Pattern Drop Shoulder Sweater.
starting 4th iteration.
GOT SHEIN Coolane Solid High Waist Flap Pocket Cargo Pants.
starting 5th iteration.
GOT DAZY Drawstring Waist Wide Leg Sweatpants.
starting 6th iteration.
GOT SHEIN Privé Twist Back Drop Shoulder Sweater.
starting 7th iteration.
GOT SHEIN Essnce Open Front Drop Shoulder Solid Cardigan.
starting 8th iteration.
GOT SHEIN Maternity Drop Shoulder Slit Hem Tee.
starting 9th iteration.
GOT SHEIN VCAY Striped Print Embroidery Detail Ruffle Hem Smock Dress.
starting 10th iteration.
GOT SHEIN BAE Lace Bustier Cami Top.
starting 11th iteration.
GOT EMERY ROSE Solid Button Detail Roll Up Hem Capri Pants.
starting 12th iteration.
GOT SHEIN Belle Criss Cross Backless Bridesmaid Dress.
start

In [32]:
output.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   product_name      119 non-null    object
 1   product_price     119 non-null    object
 2   product_discount  119 non-null    object
 3   average_rating    119 non-null    object
 4   fit_rating        119 non-null    object
 5   num_review        119 non-null    object
 6   product_tags      119 non-null    object
 7   top_3_reviews     119 non-null    object
dtypes: object(8)
memory usage: 7.6+ KB
